# Study of extreme class imbalance in AUC estimation via the random classifier

## Recap: the C statistic 

Recall that the area under the ROC curve, the ROC AUC (denoted simply as AUC below) can be shown to be equivalent to the C-statistic, ie. it has a probabilistic interpration as

$$\boxed{\mathrm{AUC} := \mathbb P(Q_1 \geq Q_0)}$$

where $Q_i = f(X)|Y=i$ is the score distribution for class $i \in \{0,1\}$ under a model $f$.

Consider a dataset $\{x_i, y_i\}_{i=1}^N$ with $n$ entries of class 1 and $N - n$ entries of class 0. Denote the index sets for these classes as $I \equiv \{i: y_i=1\}$ and $J \equiv \{j: y_j = 0\}$.

Let us write $p_i = f(x_i)$ to be the model-forecasted score for entry $x_i$. Then, the quantity

$$\boxed{\widehat{\mathrm{AUC}} := \frac{1}{n(N-n)} \sum_{i\in I}\sum_{j\in J} 1_{p_i \geq p_j}}$$

is an estimator for the real AUC.

### Example (to show that this is true)

In [3]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

In [12]:
X, y = make_classification(2000, n_features=20, class_sep=0.6, weights=(0.8,), random_state=2)

In [18]:
model = LogisticRegression().fit(X, y)

y_probs = model.predict_proba(X)[:,1]
auc_area = roc_auc_score(y, y_probs)

In [19]:
print("ROC AUC calculated as an area:", round(auc_area,5))

ROC AUC calculated as an area: 0.93089


In [20]:
n0, n1 = np.bincount(y)
total_sum = 0
for pi in y_probs[y==1]:
    for pj in y_probs[y==0]:
        if pi >= pj:
            total_sum += 1
auc_estimator = total_sum/(n0*n1)

In [22]:
print("ROC AUC calculated as an area:", round(auc_estimator,5))

ROC AUC calculated as an area: 0.93089


Notice how both results are identical.

## How far away from the actual AUC does the estimator go?

When calculating the AUC from a finite sample, one might wonder how precise is the estimator. This problem can be tackled by bootstrap.

We want to obtain some feeling for an analytical result, so let us get the simplest possible estimator: a random one, where all scores are uniformly sampled from $[0,1]$. In this context, how likely are we to get a spuriously large AUC?

In particular, we want to consider an **extreme imbalance scenario** where

1. $n$ is small (say, less than 100)

2. The ratio $n/N$ is small: $n/N \ll 1$ (say, 1% or less)

Our intuition tells us that this case should be more interesting than the big-data, balanced one: since one has only a few ($n$) points in class 1, if we are "lucky" to get high scores for all of them, the AUC will end up being high. 

We will use the AUC estimator as defined above. First, we have a trivial result: for all $i \in I$, $j \in J$, if $p_i, p_j$ iid distributed as $\mathrm{Uniform}([0,1])$ then

$$1_{p_i \geq p_j} | p_i \; \sim \;\mathrm{Bernoulli}(p_i)$$

Since a binomial variable is built from a sum of independent Bernoulli ones, we have a corollary: for all $i \in I$, 

$$\sum_{j \in J}\left. 1_{p_i \geq p_j} \right| p_i \; \sim \; \mathrm{Binomial}(N-n, p_i)$$

Now, for large $N-n$, we may use the normal approximation to the binomial, namely a $\mathrm{Binomial}(n,p)$ variable converges to a $\mathcal N(\mu=np, \sigma^2 = np(1-p))$ variable as $n$ grows. Hence

$$\sum_{j \in J}\left. 1_{p_i \geq p_j} \right| p_i \; \sim \; \mathcal N\left( (N-n)p_i,  (N-n)p_i (1-p_i) \right)$$

It follows that, for all $i\in I$,

$$Z_i |p_i := \frac{1}{N-n} \sum_{j \in J} \left.1_{p_i \geq p_j}\right| \, p_i \;\sim \; \mathcal N \left(p_i, \frac{p_i (1-p_i)}{N-n}\right)$$

This defines a set of $n$ variables $Z_i$. To obtain their marginal distribution, notice that for any $Z_i$ its PDF is given by

$$p_{Z_i}(z) = \int p(z|p_i) p(p_i) dp_i;$$

but $p_i \; \sim \; \mathrm{Uniform}([0,1])$, and hence its PDF is just the identity on $[0,1]$. Letting

$$f(x|\mu,\sigma^2) \equiv \frac{1}{\sqrt{2\pi \sigma^2}} \exp\left\{ - \frac{(x-\mu)^2}{2\sigma^2} \right\}$$

be the Gaussian PDF, we obtain

$$p_{Z_i}(z) = \int_0^1 f \left(z \left. \frac{}{} \right|\, p, \frac{p(1-p)}{N-n} \right) dp$$

For $N-n$ large (as is our hypothesis), the integrand (as a function of $p$) is basically a very sharp peak centered at $p=z$. In fact, we may approximate it as a Dirac delta function

$$f\left(z \left. \frac{}{} \right|\, p, \frac{p(1-p)}{N-n} \right) \approx \delta(z-p)$$

to obtain

$$p_{Z_i}(z) = 1_{z \in [0,1]} \quad \Rightarrow \quad Z_i \; \sim \; \mathrm{Uniform}([0,1])$$

> We have also tested this numerically - even for $n/N$ not that small this holds surprisingly well.

**If we assume all $Z_i$'s are independent among themselves**, the it means that $n \widehat{\mathrm{AUC}}$ is the sum of $n$ independent uniform variables: it follows the so-called Irwin-Hall distribution, and we have our most important result:

> Notice: we have not proven this independence assumption - it is **not obvious**, and we prove it below.

**Theorem**. Let $\widehat{\mathrm{AUC}}_\mathrm{random}$ denote the ROC AUC estimator for a *uniformly random scoring function*. Let there be $n$ instances of class 1 and $N-n$ instances of class 0, where $N\geq n$. Then

$$\boxed{n \widehat{\mathrm{AUC}}_\mathrm{random} \; \sim \; \mathrm{IrwinHall}(n)};$$

notice that this result **does not depend on $N$ explicitly**; we've only used that $N$ is large and also much larger than $n$.

As Wikipedia can tell us, if $A \sim \mathrm{IrwinHall}(n)$ then

$$\mathbb E[A] = \frac{n}{2};\qquad \mathrm{Var}(A) = \frac{n}{12};$$

for the AUC, this gives 

$$\boxed{\mathbb E[\widehat{\mathrm{AUC}}_\mathrm{random}] = \frac{1}{2};\qquad \mathrm{Var}[\widehat{\mathrm{AUC}}_\mathrm{random}] = \frac{1}{12 n}}$$

The first result is not surprising: we know that for a random scoring function the AUC should be 1/2. The second one is more surprising, and shows that as we increase $n$, we get an increasingly more precise AUC at 0.5.

We can now use this result to calculate how likely a statistical fluke is to happen: recall Chebyshev's inequality for any (square-integrable) random variable $X$:

$$\mathbb P(|X - \mu| \geq t) \leq \frac{\mathrm{Var} \, X}{t^2}$$

for our random AUC, this gives

$$\boxed{\mathbb P \left( \left|\widehat{\mathrm{AUC}}_\mathrm{random} - \frac 12 \right| \geq t \right) \leq \frac{1}{12 n t^2}}$$

**Example**: let $n = 20$:

In [26]:
n = 20
for t in [0.1, 0.2, 0.3, 0.4]:
    print(f"Probability of AUC > {0.5+t} is less than {round(100/(12*n*t**2), 2)}%")

Probability of AUC > 0.6 is less than 41.67%
Probability of AUC > 0.7 is less than 10.42%
Probability of AUC > 0.8 is less than 4.63%
Probability of AUC > 0.9 is less than 2.6%


Notice that there is a significant probability of AUC > 0.7: around 10%.

### In real-life models

For any model which is even slightly better than random, the reasoning above might be expanded: we can probably reach higher AUCs more easily than the random case. To be investigated further.

## Proof that the $Z_i$'s are independent as $N$ grows

Recall that, above, we defined

$$Z_i \equiv Z_i^{(N)} = \frac{1}{N-n} \sum_{j\in J} 1_{p_i \geq p_j}$$

We have shown that as $N$ grows this variable becomes uniform. We can actually prove something stronger: that **$Z_i$ essentially becomes $p_i$ itself!**

The intuition here is that, as $N$ grows large, the $p_j$'s basically cover the whole $[0,1]$ interval, and since $Z_i$ cares only about their aggregated values, it essencially becomes independent of them.

More precisely: let $p_J \equiv \{p_j: j \in J\}$ be a shorthand for all the scores in class 0. We will show three results:

* **Proposition 1:** $Z_i|p_J$ converges on the mean to $p_i$, ie. 

  $$\lim_{N\to\infty} \mathbb E\left[\left.Z_i^{(N)} - p_i \;\right| p_J\right] = 0$$
  
* **Corollary**: $Z_i|p_J$ converges in probability to $p_i$, ie. 

  $$\lim_{N\to\infty} \mathbb P\left(\left.|Z_i^{(N)} - p_i| \geq a \;\right| p_J\right) = 0,\qquad \forall a > 0$$
  
* **Proposition 2:** $Z_i|p_J$ converges in the mean-squared sense to $p_i$, ie. 

  $$\boxed{\lim_{N\to\infty} \mathbb E\left[\left. \left(Z_i^{(N)} - p_i \right)^2\;\right| p_J\right] = 0}$$

Since $p_i$ is independent of $p_J$, this shows that the (conditional on $p_J$) variable $Z_i$ converges to the (unconditional on $p_J$) variable $p_i$. The problem is so unbalanced that the probabilities of the majority class are essentially "integrated out", and only the scores of the minority class remain.

> In what follows, we write $N-n \equiv M$ to unclutter notation.

**Proof of Proposition 1**: by direct evaluation, using that the variable $1_{p_i \geq p_j}\,|p_j$ is Bernoulli with parameter $1-p_j$, we get

$$\begin{align*}
\mathbb E\left[\left.Z_i^{(N)} - p_i \;\right| p_J\right] &= \frac{1}{M} \sum_j \mathbb E[1_{p_i \geq p_j}\,|p_j]- \mathbb E[p_i]\\
&=\frac 1M \sum_j (1-p_j) - \frac 12\\
&= \frac 12 - \frac 1M \sum_{j\in J} p_j
\end{align*}$$

As $M$ (thus $N$) grows larger, intuitively, the $p_j$ cover $[0,1]$ and so the sum above becomes a Riemann sum. We get

$$\begin{align*}
\lim_{N\to\infty}\mathbb E\left[\left.Z_i^{(N)} - p_i \;\right| p_J\right] &= \frac 12 - \int_0^1 p\, dp \\
&=\frac 12 - \frac 12\\
&= 0
\end{align*}$$

as we wanted to prove.

**Proof of Corollary**. Recall Markov's inequality: if $X$ is an integrable, non-negative random variable, then

$$\mathbb P(X \geq t) \leq \frac{\mathbb E[X]}{t}$$

Setting $X = \mathbb E\left[\left.Z_i^{(N)} - p_i \;\right| p_J\right]$ and taking the limit gives the proof.

**Proof of Proposition 2**: we want to prove that 

$$\lim_{N\to\infty} \mathbb E\left[\left. \left(Z_i^{(N)} - p_i \right)^2\;\right| p_J\right] = 0.$$

To do that, we compute the square

$$\mathbb E\left[\left. \left(Z_i^{(N)} - p_i \right)^2\;\right| p_J\right] = \underbrace{\mathbb E\left[\left.(Z_i^{(N)})^2\,\right|p_J\right]}_\text{(*)} - 2 \underbrace{\mathbb E\left[\left. p_i Z_i^{(N)} \,\right|p_J\right]}_\text{(*)} + \underbrace{\mathbb E\left[\left.p_i^2\,\right|p_J\right]}_\text{(***)}$$

We compute one term at a time. $(***)$ is the easiest: since $p_i \,\sim\,\mathrm{Uniform}([0,1])$ is independent of $p_J$, this is just 

$$\mathbb E\left[\left.p_i^2\,\right|p_J\right] = \frac 13$$

For $(**)$, we need to explicitly compute the expectation; then, taking the limit, we will find a Riemann sum. We have

$$\begin{align*}
\mathbb E\left[\left. p_i Z_i^{(N)} \,\right|p_J\right] &= \frac 1M \sum_j \mathbb E\left[\left. p_i 1_{p_i \geq p_j}\,\right|p_j\right] \\
&= \frac 1M \sum_j \int_0^1 p 1_{p \geq p_j} \, dp = \frac 1M \sum_j \int_{p_j}^1 p\, dp \\
&= \frac 1M \sum_j \left( \frac{1-p_j^2}{2}\right)\\
&\xrightarrow[N \to \infty]{} \int_0^1 \frac{1 - p^2}{2} dp\\
&= \frac 13
\end{align*}$$

Finally, for $(*)$, the procedure is technically more involved but basically identical:

$$\begin{align*}
\mathbb E\left[\left.(Z_i^{(N)})^2\,\right|p_J\right] &= \mathbb E \left[\left. \frac{1}{M^2} \sum_{j \in J} \sum_{k\in J} 1_{p_i \geq p_j} 1_{p_i \geq p_k} \;\right|p_J\right]\\
&=\frac{1}{M^2}  \mathbb E \left[\left.\sum_{j \in J}  1_{p_i \geq p_j} + 2 \sum_j \sum_{k < j} 1_{p_i \geq p_j} 1_{p_i \geq p_k} \;\right|p_J\right]\\
\end{align*}$$

With no loss of generality, order the $p_j$'s in an ascending order, so that $p_j \geq p_k$ if $j > k$. We can then simplify the second sum by considering that $p_i$ must be larger than $\max(p_j, p_k) = p_j$. Hence

$$\begin{align*}
\mathbb E\left[\left.(Z_i^{(N)})^2\,\right|p_J\right] &= \frac{1}{M^2}\left\{
    \sum_j \mathbb E\left[\left.1_{p_i \geq p_j}\,\right|p_j\right] +
    2 \sum_j \sum_{k < j} \mathbb E\left[\left.1_{p_i \geq p_j}\,\right|p_j\right]
    \right\}\\
&= \frac{1}{M^2}\left\{\sum_j (1-p_j) + 2 \sum_j \sum_{k < j} (1-p_j)\right\}\\
&= \frac{1}{M} \left[\frac{1}{M}\sum_j (1-p_j)\right] + \frac{2}{M^2} \sum_j \sum_{k < j} (1-p_j)
\end{align*}$$

Upon taking the limit, the first term will be killed by the excess $1/M$ term in front; the second one becomes the double sum

$$\begin{align*}
\mathbb E\left[\left.(Z_i^{(N)})^2\,\right|p_J\right] &\xrightarrow[N \to \infty]{} 0 + 2 \int_0^1 dp \int_0^p dx \, (1-p)\\
&= 2 \int_0^1 dp \, p(1-p)\\
&= \frac 13.
\end{align*}$$

Hence, putting all results $(*)$ to $(***)$ together, we get

$$\lim_{N\to\infty} \mathbb E\left[\left. \left(Z_i^{(N)} - p_i \right)^2\;\right| p_J\right] = \frac 13 - 2 \cdot \frac 13 + \frac 13 = 0$$

This proves Proposition 2.

## Future outlook

Read this paper by Mohri:

https://cs.nyu.edu/~mohri/pub/area.pdf